In [26]:
# Week 7
"""
In each of the following questions monthly mortgage coupon rates should be
calculated by simply dividing the annual rate by 12. You should also assume 
that all of the securities pay monthly. You should also divide annual interest 
rates by 12 to get the corresponding monthly rate and assume monthly compounding
when computing present values.
"""

'\nIn each of the following questions monthly mortgage coupon rates should be\ncalculated by simply dividing the annual rate by 12. You should also assume \nthat all of the securities pay monthly. You should also divide annual interest \nrates by 12 to get the corresponding monthly rate and assume monthly compounding\nwhen computing present values.\n'

In [23]:
class LevelPaymentMortgage():
  def __init__(self, M0, yR, nY):
    self.mR = yR/12
    self.nM = nY*12
    self.M0 = M0
    print(self.mR, self.nM, self.M0)


  def monthlyPayment(self):
    """Returns monthly payment B using Eq. 3 """
    # return self.M0*self.mR /(1 - (1 + self.mR)**(-self.nM))
    return self.M0*self.mR/(1.-(1.+self.mR)**(-self.nM))

In [24]:
# Q1.
r_annual = 5/100
principal = 400000
n_year = 30
B = LevelPaymentMortgage(principal, r_annual, n_year)
B.monthlyPayment()
# print(B.monthlyPayment())

0.004166666666666667 360 400000


2147.2864920485595

In [55]:
class PassThroughMBS():
  def __init__(self, pM0, sM, nY, cY, ptY, preMult):
    cM = cY/12
    nM = nY*12
    ptM = ptY/12
    remainingM = pM0
    poolAgeInMonths = sM + 1

    self.monthlyPaymentList = []
    self.monthlyInterestPaidByMortgageHoldersList = []
    self.monthlyInterestPaidToInvestorsList = []
    self.monthlyPrincipalRepaymentList = []
    self.annualPrepaymentRateList = []
    self.monthlyPrepaymentList = []
    self.monthlyPrincipalPaymentList = []
    self.remainingMList = []

    while remainingM > 0.01:
      monthlyPayment = remainingM*cM/(1 - (1 + cM)**(-(nM - poolAgeInMonths + 1)))
      monthlyInterestPaidByMortgageHolders = cM*remainingM
      monthlyInterestPaidToInvestors = ptM*remainingM
      monthlyPrincipalRepayment = monthlyPayment - monthlyInterestPaidByMortgageHolders

      # PSA 
      if poolAgeInMonths > 30 :
         annualPrepaymentRate = 0.06*preMult
      else:
         annualPrepaymentRate = 0.06*poolAgeInMonths/30*preMult
      monthlyPrepayment = (remainingM - monthlyPrincipalRepayment)*(
                                     1.-(1.-annualPrepaymentRate)**(1./12.))
      monthlyPrincipalPayment = monthlyPrincipalRepayment + monthlyPrepayment
      remainingM -= monthlyPrincipalPayment
      poolAgeInMonths +=1

      self.monthlyPaymentList.append(monthlyPayment)
      self.monthlyInterestPaidByMortgageHoldersList.append(monthlyInterestPaidByMortgageHolders)
      self.monthlyInterestPaidToInvestorsList.append(monthlyInterestPaidToInvestors)
      self.monthlyPrincipalRepaymentList.append(monthlyPrincipalRepayment)
      self.annualPrepaymentRateList.append(annualPrepaymentRate)
      self.monthlyPrepaymentList.append(monthlyPrepayment)
      self.monthlyPrincipalPaymentList.append(monthlyPrincipalPayment)
      self.remainingMList.append(remainingM)

In [56]:
# Q2
poolPrincipal = 400 # Million
seasoningMonths = 0 # Don't understand
loanTermYears = 20
annualCouponRate = 6/100
passThroughRate = 5/100
prepaymentMultiplier = 1 # PSA definition , CPR, considers 100 PSA

MBS_interest = PassThroughMBS(poolPrincipal, seasoningMonths, loanTermYears,
                              annualCouponRate, passThroughRate, 
                              prepaymentMultiplier)
print(sum(MBS_interest.monthlyInterestPaidToInvestorsList))



171.17626793666105


In [58]:
# Q3 
print(sum(MBS_interest.monthlyPrepaymentList))

181.0909229122059


In [59]:
# Q4 
# what is the total amount of the prepayments if the rate of prepayments 
# increases to 200 PSA? 
prepaymentMultiplier = 2
MBS_interest = PassThroughMBS(poolPrincipal, seasoningMonths, loanTermYears,
                              annualCouponRate, passThroughRate, 
                              prepaymentMultiplier)
print(sum(MBS_interest.monthlyPrepaymentList))


268.15013083265126



In [64]:
# Q5
prepaymentMultiplier = 1
MBS_interest = PassThroughMBS(poolPrincipal, seasoningMonths, loanTermYears,
                              annualCouponRate, passThroughRate, 
                              prepaymentMultiplier)
annualRiskFreeRate = 0.045
PV  = 0 # Present value
k = 1
for payment in MBS_interest.monthlyPrincipalPaymentList:
  PV +=payment/(1 + annualRiskFreeRate/12)**k
  k +=1
print(PV)

280.0950076451088


In [65]:
# Q6
PV = 0
k = 1
for payment in MBS_interest.monthlyInterestPaidToInvestorsList:
  PV += payment/(1 + annualRiskFreeRate/12)**k
  k +=1

print(PV)

133.22776928321528


In [66]:
# Q7
avgLifeInYears = 0
k = 1
interestPayments = MBS_interest.monthlyInterestPaidToInvestorsList
totalInterst = sum(interestPayments)
for payment in interestPayments:
  avgLifeInYears +=k*payment/(12*totalInterst)
  k +=1

print(avgLifeInYears)

6.007671988639568


In [68]:
# Q8
annualRiskFreeRate = 0.035
PV35 = 0
k = 1
for payment in MBS_interest.monthlyInterestPaidToInvestorsList:
  PV35 +=payment/(1 + annualRiskFreeRate/12)**k
  k +=1
print('The present value gain due to decrease in risk free rate\n', PV35 - PV)

The present value gain due to decrease in risk free rate
 7.172361899577766


In [70]:
# Q9
prepaymentMultiplier = 1.5
MBS_interest = PassThroughMBS(poolPrincipal, seasoningMonths, loanTermYears,
                              annualCouponRate, passThroughRate, 
                              prepaymentMultiplier)

PV35_psaup = 0
k = 1
for payment in MBS_interest.monthlyInterestPaidToInvestorsList:
  PV35_psaup +=payment/(1 + annualRiskFreeRate/12)**k
  k +=1

print('Present value gain due to the changes\n', PV35_psaup - PV)


Present value gain due to the changes
 -9.578348060319783
